<h2 align=center> Facial Expression Recognition with Keras</h2>

# Data Prepossessing:

In [58]:
import numpy as np
import pandas as pd
import cv2


data = pd.read_csv('archive/train/icml_face_data.csv')

In [1]:
data.iloc[1:,0]

NameError: name 'data' is not defined

In [46]:
data.iloc[:,1].unique()

array(['Training', 'PublicTest', 'PrivateTest'], dtype=object)

In [27]:
def data_prep(data):

    
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label


In [28]:
train_image, train_label = data_prep(data[data[' Usage']=='Training'])
val_image, val_label = data_prep(data[data[' Usage']=='PrivateTest'])
test_image, test_label = data_prep(data[data[' Usage']=='PublicTest'])

In [29]:
train_images = train_image.reshape((train_image.shape[0], 48, 48, 1))
train_images = train_images/255

val_images = val_image.reshape((val_image.shape[0], 48, 48, 1))
    val_images = val_images/255

    test_images = test_image.reshape((test_image.shape[0], 48, 48, 1))
    test_images = test_images/255

# CSV to Pixel

In [ ]:
output_path =  "data/images"

if os.path.exists(output_path):
    os.system('rm -rf {}'.format(output_path))

os.system('mkdir {}'.format(output_path))

label_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

data = np.genfromtxt('archive/train/icml_face_data.csv',delimiter=',',dtype=None, encoding=None)
labels = data[1:,0].astype(np.int32)
image_buffer = data[1:,1]
images = np.array([np.fromstring(image, np.uint8, sep=' ') for image in image_buffer])
usage = data[1:,2]
dataset = zip(labels, images, usage)
usage_path = ""
for i, d in enumerate(dataset):
    if(d[-1] == "Training" or d[-1] == "PrivateTest"):
        usage_path = os.path.join(output_path, "Training")
    else:
        usage_path = os.path.join(output_path, d[-1])

    label_path = os.path.join(usage_path, label_names[d[0]])
    img = d[1].reshape((48,48))
    img_name = '%08d.jpg' % i
    img_path = os.path.join(label_path, img_name)
    if not os.path.exists(usage_path):
        os.system('mkdir {}'.format(usage_path))
    if not os.path.exists(label_path):
        os.system('mkdir {}'.format(label_path))
    cv2.imwrite(img_path, img)

    #     print('Write {}'.format(img_path))


### Task 1: Import Libraries

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import os
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from IPython.display import SVG, Image
from livelossplot import PlotLossesKeras
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.3.1


### Task 2: Plot Sample Image

In [ ]:
utils.datasets.fer.plot_example_image(plt).show()

In [ ]:
for expression in os.listdir("train/"):
    print(str(len(os.listdir('train/' +expression))) + ' ' + expression + ' images')

### Task 3: Generate Training and Validation Batches

In [ ]:
img_size=48
batch_size=64

datagen_train= ImageDataGenerator(horizontal_flip=True)
train_generator= datagen_train.flow_from_directory('train/',
                                                   target_size=(img_size, img_size),
                                                  color_mode='grayscale',
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  shuffle=True)


datagen_validation ImageDataGenerator(horizontal_flip=True)
validation_generator= datagen_validation.flow_from_directory('test/',
                                                   target_size=(img_size, img_size),
                                                  color_mode='grayscale',
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  shuffle=True)

### Task 4: Create CNN Model

![](model.png)
Inspired by Goodfellow, I.J., et.al. (2013). Challenged in representation learning: A report of three machine learning contests. *Neural Networks*, 64, 59-63. [doi:10.1016/j.neunet.2014.09.005](https://arxiv.org/pdf/1307.0414.pdf)

In [16]:
model= Sequential()

model.add(Conv2D(64, (3, 3), padding='same',input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(128, (5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(512, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(64, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(512, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

opt=Adam(lr=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization_15 (Batc (None, 48, 48, 64)        256       
_________________________________________________________________
activation_15 (Activation)   (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 24, 24, 128)       204928    
_________________________________________________________________
batch_normalization_16 (Batc (None, 24, 24, 128)      

### Task 6: Train and Evaluate Model

In [1]:
epochs = 15
step_per_epoch= train_generator.n//train_genereator.batch_size
validation_steps=validation_generator.n//validation_generator.batch_size


checkpoint= ModelCheckpoint('model_weights.h5', monitor='val_accuracy', 
                           save_weights_only=True, mode='max', verbose=1)
reduce_lr= ReduceLROnPlateau(monitor='val_loss',factor=0.1, patience=2,min_lr=0,.00001,mode='auto')

callbacks=[PlotLossesKeras(),checkpoint, reduce_lr]

history=model.fit(
    x=train_generator,
    step_per_epoch=step_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=callbacks

)

NameError: name 'train_generator' is not defined

### Task 7: Represent Model as JSON String

In [ ]:
model_json=model.to_json()
with open('model.json','w') as json_file:
    json_file.write(model_json)